<a href="https://colab.research.google.com/github/fodana/Pubmed-Trigeminal-Neuralgia-Webscrapper/blob/main/webscrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

In [ ]:
from Bio import Entrez
import pandas as pd
import numpy as np
import random
random.seed(10)

In [ ]:
#Using e-search API to find PubMed search results
def search(query):
    Entrez.email = 'example@email.com'
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmax='250000',
                            retmode='xml',
                            term=query)
    results = Entrez.read(handle)
    return results

In [ ]:
#Performing the search with the Mesh terms of Alzheimer Disease
studies = search('Trigeminal Neuralgia')
studiesIdList = studies['IdList']
len(studiesIdList)

9999

In [ ]:
#Reducing the number of papers to 5000 for better computational performance
studiesIdList = random.sample(studiesIdList, 5000)
len(studiesIdList)

5000

In [ ]:
#Using e-fetch to get details from the papers obtained from e-search
def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'example@email.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

In [ ]:
#Making a DF with article information

title_list= []
abstract_list=[]
journal_list = []
language_list =[]
pubdate_year_list = []
pubdate_month_list = []

studies = fetch_details(studiesIdList)

chunk_size = 10000
for chunk_i in range(0, len(studiesIdList), chunk_size):
  chunk = studiesIdList[chunk_i:chunk_i + chunk_size]
  papers = fetch_details(chunk)
  for i, paper in enumerate (papers['PubmedArticle']):
    title_list.append(paper['MedlineCitation']['Article']['ArticleTitle'])
    try:
      abstract_list.append(paper['MedlineCitation']['Article']['Abstract']['AbstractText'][0])
    except:
      abstract_list.append('No Abstract')
    journal_list.append(paper['MedlineCitation']['Article']['Journal']['Title'])
    language_list.append(paper['MedlineCitation']['Article']['Language'][0])
    try:
      pubdate_year_list.append(paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year'])
    except:
      pubdate_year_list.append('No Data')
    try:
      pubdate_month_list.append(paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Month'])
    except:
      pubdate_month_list.append('No Data')

df = pd.DataFrame(list(zip(
    title_list, abstract_list, journal_list, language_list, pubdate_year_list, pubdate_month_list
    )),
    columns=[
             'Title', 'Abstract', 'Journal', 'Language', 'Year','Month'
             ])
df.shape

(4992, 6)

In [ ]:
df.head(10)

,Title,Abstract,Journal,Language,Year,Month
0,Sphenopalatine ganglion pulsed radiofrequency ...,There are various facial pain syndromes includ...,Brazilian journal of anesthesiology (Elsevier),eng,2016,No Data
1,Surgical management of medically refractory tr...,The role of surgery for patients with medicall...,Current neurology and neuroscience reports,eng,2012,Apr
2,Low-level laser therapy in the management of d...,The authors analysed the effects of low-level ...,Journal of clinical laser medicine & surgery,eng,1997,No Data
3,Peculiar CADASIL phenotype in monozygotic twin...,Cerebral autosomal dominant arteriopathy with ...,European review for medical and pharmacologica...,eng,2024,Feb
4,Sleep disorders and chronic craniofacial pain:...,"Chronic craniofacial pain involves the head, f...",Sleep medicine reviews,eng,2017,Jun
5,Primary trigeminal neuralgia (algophoric deaff...,In the article the author presents his theory ...,Neurologija,eng,1990,No Data
6,Chlorphenesin for trigeminal neuralgia.,No Abstract,JAMA,eng,1973,Sep
7,Comparative study of corneal and blink reflex ...,No Abstract,Advances in neurology,eng,1983,No Data
8,[Pharmacological treatment of trigeminal neura...,Trigeminal neuralgia is a syndrome of chronic ...,Revista brasileira de anestesiologia,por,2004,Dec
9,Quality of reporting in evaluations of surgica...,There are numerous reports on the surgical tre...,Neurosurgery,eng,2003,Jul


In [ ]:
#Standardizing months
df['Month'].replace('Jan', '01', inplace=True)
df['Month'].replace('Feb', '02', inplace=True)
df['Month'].replace('Mar', '03', inplace=True)
df['Month'].replace('Apr', '04', inplace=True)
df['Month'].replace('May', '05', inplace=True)
df['Month'].replace('Jun', '06', inplace=True)
df['Month'].replace('Jul', '07', inplace=True)
df['Month'].replace('Aug', '08', inplace=True)
df['Month'].replace('Sep', '09', inplace=True)
df['Month'].replace('Oct', '10', inplace=True)
df['Month'].replace('Nov', '11', inplace=True)
df['Month'].replace('Dec', '12', inplace=True)
df['Month'].replace('No Data', np.nan, inplace=True)

In [ ]:
df.head(10)

,Title,Abstract,Journal,Language,Year,Month
0,Sphenopalatine ganglion pulsed radiofrequency ...,There are various facial pain syndromes includ...,Brazilian journal of anesthesiology (Elsevier),eng,2016,NaN
1,Surgical management of medically refractory tr...,The role of surgery for patients with medicall...,Current neurology and neuroscience reports,eng,2012,04
2,Low-level laser therapy in the management of d...,The authors analysed the effects of low-level ...,Journal of clinical laser medicine & surgery,eng,1997,NaN
3,Peculiar CADASIL phenotype in monozygotic twin...,Cerebral autosomal dominant arteriopathy with ...,European review for medical and pharmacologica...,eng,2024,02
4,Sleep disorders and chronic craniofacial pain:...,"Chronic craniofacial pain involves the head, f...",Sleep medicine reviews,eng,2017,06
5,Primary trigeminal neuralgia (algophoric deaff...,In the article the author presents his theory ...,Neurologija,eng,1990,NaN
6,Chlorphenesin for trigeminal neuralgia.,No Abstract,JAMA,eng,1973,09
7,Comparative study of corneal and blink reflex ...,No Abstract,Advances in neurology,eng,1983,NaN
8,[Pharmacological treatment of trigeminal neura...,Trigeminal neuralgia is a syndrome of chronic ...,Revista brasileira de anestesiologia,por,2004,12
9,Quality of reporting in evaluations of surgica...,There are numerous reports on the surgical tre...,Neurosurgery,eng,2003,07


Prevalence and Demographic Analysis

Symptom and Pain Management Analysis

Treatment Outcome Analysis

Risk Factor Analysis

Trends in Research and Treatment Innovations

Patient Quality of Life Analysis